In [18]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import os
import time
from qiskit.aqua.algorithms import NumPyEigensolver
from typing import List, NamedTuple
from qiskit.chemistry.transformations import FermionicTransformation, FermionicTransformationType, FermionicQubitMappingType
import pylab
import os
import time
from typing import List, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from qiskit import QuantumCircuit
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import PauliExpectation
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.converters import AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator, AerProvider
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import Statevector
from qiskit.test.mock import *
from qiskit.visualization import plot_error_map

In [ ]:
#plt.rcParams["figure.dpi"] = 75

In [11]:
#file_path = './qmolecule-data'
start = 0.1
step_size = 0.2
steps = 5
# bond_lengths = np.linspace(0.1, 3, steps)
basis_sets = ['sto3g', 'sto6g', '321g', '631g']
threshold = 0.00000001

In [4]:
# using driver to get fermionic Hamiltonian
driver_0_735_sto3g = PySCFDriver(atom="H 0.0 0.0 0.0; H 0.0 0.0 0.735" , unit=UnitsType.ANGSTROM, charge=0,
                             spin=0, basis='sto3g')
molecule_0_735_sto3g = driver_0_735_sto3g.run()
h1_0_735_sto3g = molecule_0_735_sto3g.one_body_integrals
h2_0_735_sto3g = molecule_0_735_sto3g.two_body_integrals

In [6]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1_0_735_sto3g, h2=h2_0_735_sto3g)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=threshold)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=threshold)
qubitOp_bk = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=threshold)

In [7]:
# print out qubit hamiltonian in Pauli terms and exact solution
qubit_ops = [(qubitOp_jw, 'jordan wigner'),
            (qubitOp_pa, 'parity'),
            (qubitOp_bk, 'bravyi-kitaev')]

for qubit_op, name in qubit_ops:
    print("\n --- {} ---".format(name))
    print(qubit_op)


 --- jordan wigner ---
Representation: paulis, qubits: 4, size: 15

 --- parity ---
Representation: paulis, qubits: 4, size: 15

 --- bravyi-kitaev ---
Representation: paulis, qubits: 4, size: 15


In [8]:
exact_energies  = np.empty([len(basis_sets), steps+1])
distances = np.empty(steps+1)

In [9]:
fermionic_transformation = FermionicTransformation(
            transformation=FermionicTransformationType.FULL,
            qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER,
            two_qubit_reduction=False,
            freeze_core=False)

In [13]:
print('Processing step __', end='')
for i in range(steps+1):
    print('\b\b{:2d}'.format(i), end='', flush=True)
    d = start + i*step_size/steps
    for j in range(len(basis_sets)):
        driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(d), unit=UnitsType.ANGSTROM, charge=0,
                             spin=0, basis=basis_sets[j])
        qmolecule = driver.run()
        repulsion = qmolecule.nuclear_repulsion_energy
        qubit_op, _ = fermionic_transformation.transform(driver)
        exact_solution = NumPyEigensolver(qubit_op).run()
        exact_energies[j][i] = np.real(exact_solution.eigenvalues) + repulsion
    distances[i] = d
print(' --- complete')

Processing step __ 0 1 2 3 4 5 --- complete


In [14]:
print('Distances: ', distances)
print('Energies:', exact_energies)


Distances:  [0.1  0.14 0.18 0.22 0.26 0.3  0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
Energies: [[ 2.70996077e+000  1.22996410e+000  4.29504994e-001 -5.99421221e-002
  -3.81243002e-001 -6.01803711e-001  0.00000000e+000  3.95252517e-323
   6.93340905e-310  6.93352057e-310  6.93352056e-310  6.93351917e-310
   6.93352056e-310  6.93352056e-310  6.93351918e-310  6.93351916e-310
   6.93351918e-310  6.93352057e-310  6.93351917e-310  6.93351918e-310
   6.93351916e-310  6.93351916e-310  6.93351918e-310  6.93351918e-310
   6.93352056e-310  6.93351916e-310  6.93352056e-310  6.93352058e-310
   6.93351918e-310  6.93352056e-310  6.93352057e-310  6.93352056e-310
   6.93351918e-310  6.93351917e-310  6.93351916e-310  6.93352069e-310
   6.93351918e-310  6.93351917e-310  6.93352056e-310  6.93351918e-310
   6.93351918e-310  6.93

In [20]:
for j in range(len(basis_sets)):
    pylab.plot(distances, exact_energies[j], label=basis_sets[j])
pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title('H2 Ground State Energy in different basis sets')
pylab.legend(loc='upper right');

In [ ]:
def create_qmolecule(bond_length, basis_set):
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(bond_length), unit=UnitsType.ANGSTROM, charge=0,
                             spin=0, basis=basis_set)
    molecule = driver.run()
    file_name = '_'.join(['h2',str(bond_length).replace('.','_')])
    %molecule.save(os.path.join(file_path,file_name))